In [3]:
pip install newspaper3k


  Using cached tldextract-3.1.0-py2.py3-none-any.whl (87 kB)
  Using cached feedfinder2-0.0.4.tar.gz (3.3 kB)
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13537 sha256=0193e15d0089ccc5d78a498e5d43df30a91d397dbd0b864b316108f9a8011fcf
  Stored in directory: c:\users\sajin\appdata\local\pip\cache\wheels\99\74\83\8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=a71be6c3ece227a7d1f32fdadf4728ec58995f8636b1fd6fd6ad762723dd3703
  Stored in directory: c:\users\sajin\appdata\local\pip\cache\wheels\b6\09\68\a9f15498ac02c23dde29f18745bc6a6f574ba4ab41861a3575
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398406 sha256=1c17b06873fc6a14bb0d72c6e2bd2e79cd5e96693a1f2cac5b90a3550e64c8a0
  Stored in directory: c:\users\sajin\appdata\local\pip\cache\wheels\1f\7e\0c\54f3b0f5164278677899f2db08f2b07943ce2d024a3c862afb
  Created wheel for sgmllib

In [4]:
from newspaper import Article
import random
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import requests
import bs4
import warnings
warnings.filterwarnings('ignore')

In [5]:
nltk.download('punkt',quiet=True)
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SAJIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
res=requests.get('https://www.who.int/emergencies/diseases/novel-coronavirus-2019/question-and-answers-hub/q-a-detail/coronavirus-disease-covid-19')

In [7]:
soup=bs4.BeautifulSoup(res.text,'lxml')
sent_token=[]
word_token=[]
for i in soup.select('.sf-accordion__content'):
    
    corpus=i.text
    sent_token_list = nltk.sent_tokenize(corpus)
    word_token_list = nltk.word_tokenize(corpus)
    sentence=nltk.sent_tokenize(corpus)
    word_token=word_token+word_token_list
    sent_token=sent_token+sent_token_list
   
print(word_token)
print(sent_token)

['COVID-19', 'is', 'the', 'disease', 'caused', 'by', 'a', 'new', 'coronavirus', 'called', 'SARS-CoV-2', '.', 'WHO', 'first', 'learned', 'of', 'this', 'new', 'virus', 'on', '31', 'December', '2019', ',', 'following', 'a', 'report', 'of', 'a', 'cluster', 'of', 'cases', 'of', '‘', 'viral', 'pneumonia', '’', 'in', 'Wuhan', ',', 'People', '’', 's', 'Republic', 'of', 'China', '.', 'The', 'most', 'common', 'symptoms', 'of', 'COVID-19', 'areFeverDry', 'coughFatigueOther', 'symptoms', 'that', 'are', 'less', 'common', 'and', 'may', 'affect', 'some', 'patients', 'include', ':', 'Loss', 'of', 'taste', 'or', 'smell', ',', 'Nasal', 'congestion', ',', 'Conjunctivitis', '(', 'also', 'known', 'as', 'red', 'eyes', ')', 'Sore', 'throat', ',', 'Headache', ',', 'Muscle', 'or', 'joint', 'pain', ',', 'Different', 'types', 'of', 'skin', 'rash', ',', 'Nausea', 'or', 'vomiting', ',', 'Diarrhea', ',', 'Chills', 'or', 'dizziness', '.', 'Symptoms', 'of', 'severe', 'COVID‐19', 'disease', 'include', ':', 'Shortness'

In [8]:
lemmer = nltk.stem.WordNetLemmatizer()

In [9]:
def lemmer_tokens(tokens):
    return[lemmer.lemmatize(token) for token in tokens]

In [10]:
remove_punct = dict((ord(punct), None) for punct in string.punctuation)


In [11]:
def lemmer_normalize(text):
    return lemmer_tokens(nltk.word_tokenize(text.lower().translate(remove_punct)))

In [12]:
GREETING_INPUT = ('hi','hai', 'hello', 'hey', 'greetings my friend')

GREETING_RESPONSE = ['Are you talking to me?', 'Nice to hear you', 'Hi my friend!']

In [13]:
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUT:
            return random.choice(GREETING_RESPONSE)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
def response(user_response):
    rat_response = ''

    TfidfVec = TfidfVectorizer(tokenizer=lemmer_normalize, stop_words='english')

    tfidf = TfidfVec.fit_transform(sent_token)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if req_tfidf == 0:
        rat_response = rat_response + "I don't understand,Can you say it again!"
        return rat_response
    else:
        rat_response = rat_response + sent_token[idx]
        return rat_response

In [ ]:
flag = True

print("Chat Bot: Ask me anythng about Covid-19.\
I will answer your queries about Space. If you want to exit, type Bye!")
while flag is True:
    user_response = input()
    user_response = user_response.lower()
    if user_response != 'bye':
        if user_response == 'thanks' or user_response == 'thank you':
            flag = False
            print("Chat Bot: You are welcome..")
        else:
            if greeting(user_response) is not None:
                print("Chat Bot: " + greeting(user_response))
            else:
                sent_token.append(user_response)
                word_token = word_token + nltk.word_tokenize(user_response)
                final_words = list(set(word_token))
                print("Chat Bot: ", end="")
                print(response(user_response))
                sent_token.remove(user_response)
    else:
        flag = False
        print("Chat Bot: Bye! take care..")

Chat Bot: Ask me anythng about Covid-19.I will answer your queries about Space. If you want to exit, type Bye!
hi
Chat Bot: hey dude, nice to hear you
What is COVID-19?
Chat Bot: After exposure to someone who has COVID-19, do the following: Call your health care provider or COVID-19 hotline to find out where and when to get a test.
What are the symptoms of COVID-19?
Chat Bot: 

Some people who have had COVID-19, whether they have needed hospitalization or not, continue to experience symptoms, including fatigue, respiratory and neurological symptoms.
